# Initialisation

In [ ]:
%matplotlib ipympl
from matplotlib import pyplot as plt
import numpy as np
#import scipy
from scipy.special import zeta
from time import time
from astropy import cosmology as cosmo
from astropy import units as u
from astropy import constants as c
from astropy.visualization import quantity_support
quantity_support()

# Models

## Planck18

In [ ]:
Planck18 = cosmo.Planck18

eta = 6e-10
Y = .24
neutrinos = (7/8)*Planck18.Neff*(4/11)**(4/3)

z_p18 = np.logspace(-2, 4, 301)

t_p18 = Planck18.age(z_p18).to_value(u.yr)
Or_p18 = Planck18.Ogamma(z_p18) * (1+neutrinos)
Ob_p18 = Planck18.Ob(z_p18)
Om_p18 = Planck18.Om(z_p18)
ODE_p18 = Planck18.Ode(z_p18)

n_H_p18 = (1-Y) * Ob_p18*Planck18.critical_density(z_p18) / c.m_p
y_p18 = (2/n_H_p18) * np.power(2*np.pi*c.m_e*c.k_B*Planck18.Tcmb(z_p18)/c.h**2, 1.5) * np.exp(-13.6*u.eV/c.k_B/Planck18.Tcmb(z_p18))
x0 = np.linspace(1e-12, 1-1e-12, 1001)
y0 = x0**2/(1-x0)
x_p18 = np.interp(y_p18, y0, x0)
t_coll_p18 = (1/x_p18/n_H_p18/c.sigma_T/c.c).to_value(u.yr)
t_cmb_p18 = np.min(t_p18[t_coll_p18 > t_p18])

rho_c_p18 = Planck18.critical_density(z_p18).to_value(u.Msun/u.Mpc**3)


## LwDM (LCDM and onion)

In [ ]:
class LwDM_model(object):

    def __init__(self, Omega_DM, w_DM, Omega_L,
                 H0=70*u.km/u.s/u.Mpc,
                 Tcmb0=0*u.K, Neff=3, eta=6e-10, Y=.24,
                 z_min=1e-3, z_max=2e5,
                ):
        '''
        Lambda wDM class.
        '''
        
        # Density parameters
        
        self.Om0 = Omega_DM
        #self.w_DM = w_DM
        self.Ode0 = Omega_L
        Omega_k = 1 - Omega_DM - Omega_L
        self.Ok0 = Omega_k
                
        self.H0 = H0
        self.Tcmb0 = Tcmb0
        self.Neff = Neff
        self.set_ordinary_densities(eta, Y)

        # Expansion history
        
        step = np.log10(1 + z_min)
        log_1zmax = np.log10(1 + z_max)
        a = np.logspace(0, -log_1zmax, int(log_1zmax/step))
        self.z = 1/a[1:] - 1
        a_mid = np.sqrt(a[1:]*a[:-1])
        da = a[:-1] - a[1:]
        a_dot = np.sqrt(Omega_DM*a_mid**(-1-3*w_DM) + Omega_L*a_mid**2 + Omega_k)
        self.HH0 = a_mid / a_dot

        # Cosmic distances
        
        self.Dc_DH = np.cumsum(da/self.HH0)

        if np.abs(Omega_k) < 1e-6:
            #print(f'{Omega_k} is flat')
            self.DM_DH = self.Dc_DH
        elif Omega_k < 0:
            #print(f'{Omega_k} is closed')
            r = np.abs(np.sin(self.Dc_DH*np.sqrt(-Omega_k)))
            self.DM_DH = r/np.sqrt(-Omega_k)
        else:
            #print(f'{Omega_k} is open')
            r = np.sinh(self.Dc_DH*np.sqrt(Omega_k))
            self.DM_DH = r/np.sqrt(Omega_k)

        self.DL_DH = self.DM_DH*(1+self.z)


            
    def set_ordinary_densities(self, eta, Y):

        # photons
        rho_gamma0 = 4*c.sigma_sb * self.Tcmb0**4 / c.c**3
        self.Ogamma0 = (rho_gamma0 * 8*np.pi*c.G / 3/self.H0**2).to_value(u.dimensionless_unscaled)

        # masless neutrinos
        neutrinos = (7/8)*self.Neff*(4/11)**(4/3)
        self.Or0 = self.Ogamma0 * (1 + neutrinos)

        # (non-relativistic) matter
        mu0 = ((1 - self.Ok0) / self.Or0 - 3) / 2
        t0_teq = np.sqrt(5*mu0**4 / (3+2*mu0))

            
    def w(self, z):
        return(-1.0)
    
    def fit_mu(self, z, mu, err_mu):
        mu_model = 5*np.log10(np.interp(z, self.z, self.DL_DH)/1e-5) # units of Mpc
        
        posterior_error = 1/np.sum((1/err_mu)**2)
        best_norm = np.sum((mu-mu_model)/err_mu**2) * posterior_error
        self.chi2_mu = np.mean(((mu-mu_model-best_norm)/err_mu)**2)
        self.c_over_H0 = 10**(best_norm/5)
        self.H0 = 3e5/self.c_over_H0
        #print(f'c/H0 = {best_norm:.3g} Mpc, H0 = {3e5/best_norm:.4g} km/s/Mpc [{3e5/10**((best_norm+z.size*posterior_error)/5):.6g}, {3e5/10**((best_norm-z.size*posterior_error)/5):.6g}]')

            
    def fit_theta_BAO(self, z, theta, err_theta):
        theta_model = 180/np.pi / np.interp(z, self.z, self.DM_DH)
        self.RS_DH = np.sum((theta*theta_model)/err_theta**2) / (np.sum((theta_model/err_theta)**2))
        self.chi2_theta_BAO = np.mean(((theta - self.RS_DH*theta_model)/err_theta)**2)


In [ ]:
models = {}
#models[r'open $R=ct$ (Milne)'] = (LwDM_model(0, 0, 0), 'r:')
#models['flat $R=ct$ (Melia)'] = (LwDM_model(1, -1/3, 0), 'b:')
models[r'flat $\Lambda$CDM'] = (LwDM_model(.31115, 0, .68885), 'r--')
#models[r'closed $R=ct$'] = (LwDM_model(2, -1/3, 0), 'k-')
#models[r'closed $R=\pi{ct}$'] = (LwDM_model(1 + 1/np.pi**2, -1/3, 0), 'k--')
#models[r'$R=4.26ct$'] = (LwDM_model(1+1/4.26**2, -1/3, 0), 'k--')
Omega_U = 1.05
models[r'This work'] = (LwDM_model(Omega_U, -1/3, 0), 'b-')


In [ ]:
neutrinos = (7/8)*Planck18.Neff*(4/11)**(4/3)
Planck18.Ogamma0, Planck18.Onu0, Planck18.Om0, Planck18.Otot0, Planck18.Ode0, Planck18.Om0+Planck18.Onu0+Planck18.Ogamma0, Planck18.H0, 1-Planck18.Ode0

## Coasting

Parameters:

In [ ]:
eta = 6e-10
#eta = 6.13e-10

H0 = 70 * u.km/u.s/u.Mpc
#H0 = Planck18.H0

Ok = -.05
print('t0', (1/H0).to_value(u.Gyr), 'Gyr')

# photons
rho_gamma0 = 4*c.sigma_sb * Planck18.Tcmb0**4 / c.c**3
Ogamma0 = (rho_gamma0 * 8*np.pi*c.G / 3/H0**2).to_value(u.dimensionless_unscaled)

# masless neutrinos
neutrinos = (7/8)*Planck18.Neff*(4/11)**(4/3)
Or0 = Ogamma0 * (1 + neutrinos)

# (non-relativistic) matter
mu0 = ((1 - Ok) / Or0 - 3) / 2
t0_teq = np.sqrt(5*mu0**4 / (3+2*mu0))
print('Omega_r', Or0, Planck18.Ogamma0*(1+neutrinos))

print('mu, zeq', mu0, t0_teq - 1, Planck18.Om0/Planck18.Ogamma0)

Om0 = mu0 * Or0
rho_m0 = Om0 * 3*H0**2/(8*np.pi*c.G)
print('Omega_m', Om0, mu0 / (3 + 2 * mu0) * (1-Ok))

n_gamma0 = 16*np.pi*zeta(3) * (c.k_B*Planck18.Tcmb0 / c.h/c.c)**3
rho_b0 = eta * c.m_p * n_gamma0
Ob0 = (rho_b0 * 8*np.pi*c.G / 3/H0**2).to_value(u.dimensionless_unscaled)
print('Omega_b', Ob0, Planck18.Ob0)

#eta_m = (rho_m0 / rho_r0).to_value(u.dimensionless_unscaled)
#print(eta_m)

Evolution:

In [ ]:
mu = np.logspace(-4, 0, 501) * mu0

t_teq = np.sqrt(5*mu**4 / (3+2*mu))

z_model = t0_teq / t_teq - 1
t_model = (1/H0 / (1+z_model)).to_value(u.yr)

Or_model = (1 - Ok) / (3 + 2 * mu)
Om_model = (1 - Ok) * mu / (3 + 2 * mu)
Ob_model = Ob0/Om0 * Om_model
ODE_model = 2*Or_model + Om_model

rho_c_model = 3 / 8/np.pi/c.G / (t_model*u.yr)**2
Tcmb_model = np.power(c.c**3/(4*c.sigma_sb) * Or_model * rho_c_model / (1+neutrinos), 1/4)

n_H_model = (1-Y) * Ob_model*rho_c_model / c.m_p
y_model = (2/n_H_model) * np.power(2*np.pi*c.m_e*c.k_B*Tcmb_model/c.h**2, 1.5) * np.exp(-13.6*u.eV/c.k_B/Tcmb_model)
x0 = np.linspace(0, 1-1e-6, 1001)
y0 = x0**2/(1-x0)
x_model = np.interp(y_model, y0, x0)
t_coll_model = (1/x_model/n_H_model/c.sigma_T/c.c).to_value(u.yr)
t_cmb_model = np.min(t_model[t_coll_model > t_model])
z_cmb_model = (1/H0).to_value(u.yr) / t_cmb_model

rho_c_model = rho_c_model.to_value(u.Msun/u.Mpc**3)


In [ ]:
t_cmb_model

# Plots

In [ ]:
fig_name = 'evolution'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(8, 8))
axes = fig.subplots(nrows=2, ncols=1, squeeze=False,
                    sharex='col', sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )
alpha = 1

ax = axes[0, 0]
ax.set_ylabel(r'density $\rho$ [M$_\odot$ Mpc$^{-3}$]')

def mass_to_E(m):
    return((m*c.c**2 * u.Msun/u.Mpc**3).to_value(u.GeV/u.cm**3))
def E_to_mass(E):
    return((E/c.c**2 * u.GeV/u.cm**3).to_value(u.Msun/u.Mpc**3))
secax = ax.secondary_yaxis('right', functions=(mass_to_E, E_to_mass))
secax.set_ylabel(r'energy density $\rho c^2$ [GeV cm$^{-3}$]')


ax.set_yscale('log')
ax.set_xscale('log')

l1, = ax.plot(t_p18, ODE_p18*rho_c_p18, 'r--', alpha=alpha, label='dark energy')
l2, = ax.plot(t_p18, (Om_p18-Ob_p18)*rho_c_p18, 'k--', alpha=alpha, label='dark matter')
l3, = ax.plot(t_p18, Ob_p18*rho_c_p18, 'c--', alpha=alpha, label='baryons (atoms)')
l4, = ax.plot(t_p18, Or_p18*rho_c_p18/(1+neutrinos), 'y--', alpha=alpha, label='photons')
first_legend = ax.legend(handles=[l1, l2, l3, l4], title=r'$\Lambda$CDM', loc='upper left', bbox_to_anchor=(0.45, 0.5, 0.5, 0.5))
ax.add_artist(first_legend)

l1, = ax.plot(t_model, ODE_model*rho_c_model, 'r-', alpha=alpha, label='dark energy')
l2, = ax.plot(t_model, (Om_model-Ob_model)*rho_c_model, 'k-', alpha=alpha, label='dark matter')
l3, = ax.plot(t_model, Ob_model*rho_c_model, 'c-', alpha=alpha, label='baryons (atoms)')
l4, = ax.plot(t_model, Or_model*rho_c_model/(1+neutrinos), 'y-', alpha=alpha, label='photons')
ax.legend(handles=[l1, l2, l3, l4], title='This work', loc='upper right', bbox_to_anchor=(0.5, 0.5, 0.5, 0.5))


ax = axes[1, 0]
ax.set_ylabel('ionisation fraction $x$')

ax.plot(t_p18, x_p18, 'r--', alpha=alpha, label=r'$\Lambda$CDM: $t_{\rm CMB} \sim$'+f'{t_cmb_p18:g} yr')
ax.plot(t_model, x_model, 'k-', alpha=alpha, label=r'This work: $t_{\rm CMB} \sim$'+f'{t_cmb_model:g} yr')

ax.scatter(t_cmb_p18, np.interp(t_cmb_p18, t_p18[::-1], x_p18[::-1]), marker='d', facecolors='none', edgecolors='r')
ax.plot(t_cmb_model, np.interp(t_cmb_model, t_model, x_model), 'kd')

ax.legend()

ax.set_xscale('log')
ax.set_xlabel('cosmic time $t$ [yr]')


for ax in axes.flatten():
    ax.tick_params(which='both', direction='in')
    ax.grid(alpha=.5)
    #ax.axvline(t_cmb_p18, c='r', ls='--')
    #ax.axvline(t_cmb_model, c='k', ls='-')
fig.set_tight_layout(True)
plt.show()
plt.savefig('evolution.pdf')

## Density

In [ ]:
fig_name = 'density_comparison'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(8, 5))
axes = fig.subplots(nrows=1, ncols=1, squeeze=False,
                    sharex='col', sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )
alpha = 1

ax = axes[0, 0]
#ax.set_ylabel(r'density $\rho$')
ax.set_ylabel(r'density $\rho$ [M$_\odot$ Mpc$^{-3}$]')

def mass_to_E(m):
    return((m*c.c**2 * u.Msun/u.Mpc**3).to_value(u.GeV/u.cm**3))
def E_to_mass(E):
    return((E/c.c**2 * u.GeV/u.cm**3).to_value(u.Msun/u.Mpc**3))
secax = ax.secondary_yaxis('right', functions=(mass_to_E, E_to_mass))
secax.set_ylabel(r'energy density $\rho c^2$ [GeV cm$^{-3}$]')


#ax.set_ylabel(r'density $\rho(t) / \rho_c(t_0) = (8\pi G\rho)/(3H_0^2)$')
#ax.set_ylabel(r'$\rho(t) / \rho_c(t_0) = (8\pi G\rho)/(3H_0^2)$')
#ax.set_ylim(3e-5, 3e-1)
#ax.set_ylim(3e-5, 3e12)
ax.set_yscale('log')
ax.set_xscale('log')

'''
'''
l1, = ax.plot(t_p18, ODE_p18*rho_c_p18, 'r--', alpha=alpha, label='dark energy')
l2, = ax.plot(t_p18, (Om_p18-Ob_p18)*rho_c_p18, 'k--', alpha=alpha, label='dark matter')
l3, = ax.plot(t_p18, Ob_p18*rho_c_p18, 'c--', alpha=alpha, label='baryons (atoms)')
l4, = ax.plot(t_p18, Or_p18*rho_c_p18/(1+neutrinos), 'y--', alpha=alpha, label='photons')
first_legend = ax.legend(handles=[l1, l2, l3, l4], title=r'$\Lambda$CDM', loc='upper left', bbox_to_anchor=(0.45, 0.5, 0.5, 0.5))
ax.add_artist(first_legend)

l1, = ax.plot(t_model, ODE_model*rho_c_model, 'r-', alpha=alpha, label='dark energy')
l2, = ax.plot(t_model, (Om_model-Ob_model)*rho_c_model, 'k-', alpha=alpha, label='dark matter')
l3, = ax.plot(t_model, Ob_model*rho_c_model, 'c-', alpha=alpha, label='baryons (atoms)')
l4, = ax.plot(t_model, Or_model*rho_c_model/(1+neutrinos), 'y-', alpha=alpha, label='photons')
ax.legend(handles=[l1, l2, l3, l4], title='This work', loc='upper right', bbox_to_anchor=(0.5, 0.5, 0.5, 0.5))

'''
ax.plot(z_p18, Planck18.Tcmb(z_p18), 'r--', alpha=alpha, label=r'$\Lambda$CDM radiation')
#ax.plot(z_p18, Tcmb_p18, 'r-', alpha=alpha, label=r'$\Lambda$CDM radiation')
ax.plot(z_model, Tcmb_model, 'r-', alpha=alpha, label='coasting radiation')
#
ax.plot(z_p18, Or_p18, 'r--', alpha=alpha, label=r'$\Lambda$CDM radiation')
ax.plot(z_model, Or_model, 'r-', alpha=alpha, label='coasting radiation')
'''


#fig.legend(ncol=2, title=['a', 'n'])


ax.set_xscale('log')
#ax.set_xlim(3e-1, 3e3)
#ax.set_xlim(3e4, 3e10)
ax.set_xlabel('t [yr]')

for ax in axes.flatten():
    ax.tick_params(which='both', direction='in')
    ax.grid(alpha=.5)
fig.set_tight_layout(True)
plt.show()
plt.savefig('density.pdf')

## CMB

In [ ]:
fig_name = 'CMB'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(8, 5))
axes = fig.subplots(nrows=1, ncols=1, squeeze=False,
                    sharex='col', sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )

alpha = 1

ax = axes[0, 0]
ax.set_ylabel('ionisation fraction $x$')

ax.plot(t_p18, x_p18, 'r--', alpha=alpha, label=r'$\Lambda$CDM: $t_{\rm CMB} \sim$'+f'{t_cmb_p18:g} yr')
ax.plot(t_model, x_model, 'k-', alpha=alpha, label=r'This work: $t_{\rm CMB} \sim$'+f'{t_cmb_model:g} yr')

ax.scatter(t_cmb_p18, np.interp(t_cmb_p18, t_p18[::-1], x_p18[::-1]), marker='d', facecolors='none', edgecolors='r')
ax.plot(t_cmb_model, np.interp(t_cmb_model, t_model, x_model), 'kd')

ax.legend()

ax.set_xscale('log')
#ax.set_xlim(3e4, 3e10)
ax.set_xlabel('cosmic time $t$ [yr]')

for ax in axes.flatten():
    ax.tick_params(which='both', direction='in')
    ax.grid(alpha=.5)
fig.set_tight_layout(True)
plt.show()
plt.savefig('CMB.pdf')

In [ ]:
fig_name = 'CMB_all'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(5, 10))
axes = fig.subplots(nrows=4, ncols=1, squeeze=False,
                    sharex='col', sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )

alpha = 1

ax = axes[0, 0]
ax.set_ylabel(r'radiation temperature $T$ [K]')
ax.set_ylim(.3, 3e5)
ax.set_yscale('log')

ax.plot(t_p18, Planck18.Tcmb(z_p18), 'r--', alpha=alpha, label=r'$\Lambda$CDM')
ax.plot(t_model, Tcmb_model, 'k-', alpha=alpha, label='coasting')

ax = axes[1, 0]
ax.set_ylabel('hydrogen density $n_H$ [cm$^{-3}$]')
ax.set_yscale('log')
#ax.set_ylim(3e-2, 3e10)

ax.plot(t_p18, n_H_p18.to_value(u.cm**-3), 'r--', alpha=alpha, label=r'$\Lambda$CDM')
ax.plot(t_model, n_H_model.to_value(u.cm**-3), 'k-', alpha=alpha, label='coasting')


ax = axes[2, 0]
ax.set_ylabel('ionisation fraction $x$')
#ax.set_yscale('log')
#ax.set_ylim(3e-2, 3e10)

ax.plot(t_p18, x_p18, 'r--', alpha=alpha, label=r'$\Lambda$CDM')
ax.plot(t_model, x_model, 'k-', alpha=alpha, label='coasting')


ax = axes[3, 0]
ax.set_ylabel('collision timescale $t_{coll}$ [yr]')
ax.set_yscale('log')
ax.set_ylim(3e-2, 3e11)

ax.plot(t_p18, (1/x_p18/n_H_p18/c.sigma_T/c.c).to_value(u.yr), 'r--', alpha=alpha, label=r'$\Lambda$CDM')
ax.plot(t_model, (1/x_model/n_H_model/c.sigma_T/c.c).to_value(u.yr), 'k-', alpha=alpha, label='coasting')


ax.set_xscale('log')
#ax.set_xlim(3e4, 3e10)
ax.set_xlabel('cosmic time $t$ [yr]')

for ax in axes.flatten():
    ax.tick_params(which='both', direction='in')
    ax.grid(alpha=.5)
    ax.axvline(t_cmb_p18, c='r', ls='--')
    ax.axvline(t_cmb_model, c='k', ls='-')
fig.set_tight_layout(True)
plt.show()

# OLD STUFF

## Data

In [ ]:
SN_z, SN_err_z, SN_mu, SN_err_mu = np.loadtxt('data/Pantheon+_Data.dat', usecols=(2, 3, 10, 11), unpack=True, skiprows=1)
GRB1 = np.loadtxt('data/GRB_RNN.dat', skiprows=1)
GRB2 = np.loadtxt('data/GRB_RNN_2.dat', skiprows=1)
GRB_z = np.concatenate([GRB1[:, 0], GRB1[:, 3], GRB1[:, 6], GRB2[:, 0]])
GRB_mu = np.concatenate([GRB1[:, 1], GRB1[:, 4], GRB1[:, 7], GRB2[:, 1]])
GRB_err_mu = np.concatenate([GRB1[:, 2], GRB1[:, 5], GRB1[:, 8], GRB2[:, 2]])
QSO_z, QSO_mu, QSO_err_mu = np.loadtxt('data/table3QSO.dat', usecols=(3, 11, 12), unpack=True, skiprows=1)
#QSO_err_mu *= 2

def Dl_Mpc(mu):
    return 1e-5 * 10**(.2*mu)

#SN_Dl = Dl_Mpc(SN_mu)
#GRB_Dl = Dl_Mpc(GRB_mu)
#QSO_Dl = Dl_Mpc(QSO_mu)


all_z = np.concatenate([SN_z, QSO_z, GRB_z])
all_mu = np.concatenate([SN_mu, QSO_mu, GRB_mu])
all_err_mu = np.concatenate([SN_err_mu, QSO_err_mu, GRB_err_mu])

#all_z = SN_z
#all_mu = SN_mu
#all_err_mu = SN_err_mu
#all_z = QSO_z
#all_mu = QSO_mu
#all_err_mu = QSO_err_mu
#all_z = GRB_z
#all_mu = GRB_mu
#all_err_mu = GRB_err_mu


In [ ]:
thetaBAO = np.array([
    0.11,19.8,3.26,
    0.235,9.06,0.23,
    0.365,6.33,0.22,
    0.45,4.77,0.17,
    0.47,5.02,0.25,
    0.49,4.99,0.21,
    0.51,4.81,0.17,
    0.53,4.29,0.30,
    0.55,4.25,0.25,
    0.57,4.59,0.36,
    0.59,4.39,0.33,
    0.61,3.85,0.31,
    0.63,3.90,0.43,
    0.65,3.55,0.16,
    2.225,1.77,0.31])

theta_z = thetaBAO[::3]
theta_z_err = np.array([0.005,0.035,0.025,0.01,0.005,0.01,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.01,0.025])
theta_angle_deg = thetaBAO[1::3]
theta_angle_err = thetaBAO[2::3]
theta_relative_err = thetaBAO[2::3] / theta_angle_deg
theta_DM = 180/np.pi/theta_angle_deg

## Comparison

In [ ]:
def plot_DL_DH_obs(ax, z, mu, err_mu, **kwargs):
    #scale = 3e5 * z
    scale = z
    y = Dl_Mpc(mu) / scale
    dy_low = y-Dl_Mpc(mu-err_mu) / scale
    dy_high = Dl_Mpc(mu+err_mu) / scale - y
    ax.errorbar(z, y, yerr=(dy_low, dy_high), fmt='.', **kwargs)

In [ ]:
fig_name = 'DL'

plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(15, 12))
axes = fig.subplots(nrows=4, ncols=2, squeeze=False,
                    sharex=True, sharey=True,
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )

axes[0, 0].set_title('Individual fits to $H_0$')
axes[0, 1].set_title('Global fit to $H_0$')


ax = axes[0, 0]
ax.set_ylabel(r'$D_L(z)/z$ [Mpc]')
ax.set_ylim(0, 2.1e4)
#ax.set_yscale('log')

plot_DL_DH_obs(ax, SN_z, SN_mu, SN_err_mu, c='c', alpha=.1, label='SNe')
plot_DL_DH_obs(ax, QSO_z, QSO_mu, QSO_err_mu, c='y', alpha=.25, label='QSO')
plot_DL_DH_obs(ax, GRB_z, GRB_mu, GRB_err_mu, c='r', alpha=.25, label='SNe')
for model_name in models:
    model, style = models[model_name]
    model.fit_mu(all_z, all_mu, all_err_mu)
    ax.plot(model.z, model.DL_DH*model.c_over_H0/model.z, style,
            label=f'{model_name}:  $H_0$={model.H0:.2f};' + r' $\sqrt{\frac{\chi^2}{N}}$' + f'={np.sqrt(model.chi2_mu):.2f}')
ax.legend()

ax = axes[0, 1]
plot_DL_DH_obs(ax, SN_z, SN_mu, SN_err_mu, c='c', alpha=.1, label='SNe')
plot_DL_DH_obs(ax, QSO_z, QSO_mu, QSO_err_mu, c='y', alpha=.25, label='QSO')
plot_DL_DH_obs(ax, GRB_z, GRB_mu, GRB_err_mu, c='r', alpha=.25, label='SNe')
for model_name in models:
    model, style = models[model_name]
    model.fit_mu(all_z, all_mu, all_err_mu)
    mu_model = 5*np.log10(np.interp(all_z, model.z, model.DL_DH)*model.c_over_H0/1e-5)
    chi = np.sqrt(np.nanmean(((all_mu - mu_model)/all_err_mu)**2))
    ax.plot(model.z, model.DL_DH*model.c_over_H0/model.z, style,
            label=f'$H_0$={model.H0:.2f};' + r' $\sqrt{\frac{\chi^2}{N}}$' + f'={chi:.2f}')
ax.legend()


ax = axes[1, 0]
ax.set_ylabel(r'$D_L(z)/z$ [Mpc]')

plot_DL_DH_obs(ax, SN_z, SN_mu, SN_err_mu, c='c', alpha=.1, label='SNe')
for model_name in models:
    model, style = models[model_name]
    model.fit_mu(SN_z, SN_mu, SN_err_mu)
    ax.plot(model.z, model.DL_DH*model.c_over_H0/model.z, style,
            label=f'$H_0$={model.H0:.2f};' + r' $\sqrt{\frac{\chi^2}{N}}$' + f'={np.sqrt(model.chi2_mu):.2f}')
ax.legend()

ax = axes[1, 1]
plot_DL_DH_obs(ax, SN_z, SN_mu, SN_err_mu, c='c', alpha=.1, label='SNe')
for model_name in models:
    model, style = models[model_name]
    model.fit_mu(all_z, all_mu, all_err_mu)
    mu_model = 5*np.log10(np.interp(SN_z, model.z, model.DL_DH)*model.c_over_H0/1e-5)
    chi = np.sqrt(np.nanmean(((SN_mu - mu_model)/SN_err_mu)**2))
    ax.plot(model.z, model.DL_DH*model.c_over_H0/model.z, style,
            label=f'$H_0$={model.H0:.2f};' + r' $\sqrt{\frac{\chi^2}{N}}$' + f'={chi:.2f}')
ax.legend()


ax = axes[2, 0]
ax.set_ylabel(r'$D_L(z)/z$ [Mpc]')

plot_DL_DH_obs(ax, QSO_z, QSO_mu, QSO_err_mu, c='y', alpha=.25, label='QSO')
for model_name in models:
    model, style = models[model_name]
    model.fit_mu(QSO_z, QSO_mu, QSO_err_mu)
    ax.plot(model.z, model.DL_DH*model.c_over_H0/model.z, style,
            label=f'$H_0$={model.H0:.2f};' + r' $\sqrt{\frac{\chi^2}{N}}$' + f'={np.sqrt(model.chi2_mu):.2f}')
ax.legend()

ax = axes[2, 1]
plot_DL_DH_obs(ax, QSO_z, QSO_mu, QSO_err_mu, c='y', alpha=.25, label='QSO')
for model_name in models:
    model, style = models[model_name]
    model.fit_mu(all_z, all_mu, all_err_mu)
    mu_model = 5*np.log10(np.interp(QSO_z, model.z, model.DL_DH)*model.c_over_H0/1e-5)
    chi = np.sqrt(np.nanmean(((QSO_mu - mu_model)/QSO_err_mu)**2))
    ax.plot(model.z, model.DL_DH*model.c_over_H0/model.z, style,
            label=f'$H_0$={model.H0:.2f};' + r' $\sqrt{\frac{\chi^2}{N}}$' + f'={chi:.2f}')
ax.legend()


ax = axes[3, 0]
ax.set_ylabel(r'$D_L(z)/z$ [Mpc]')

plot_DL_DH_obs(ax, GRB_z, GRB_mu, GRB_err_mu, c='r', alpha=.25, label='GRB')
for model_name in models:
    model, style = models[model_name]
    model.fit_mu(GRB_z, GRB_mu, GRB_err_mu)
    ax.plot(model.z, model.DL_DH*model.c_over_H0/model.z, style,
            label=f'$H_0$={model.H0:.2f};' + r' $\sqrt{\frac{\chi^2}{N}}$' + f'={np.sqrt(model.chi2_mu):.2f}')
ax.legend()

ax = axes[3, 1]
plot_DL_DH_obs(ax, GRB_z, GRB_mu, GRB_err_mu, c='r', alpha=.25, label='GRB')
for model_name in models:
    model, style = models[model_name]
    model.fit_mu(all_z, all_mu, all_err_mu)
    mu_model = 5*np.log10(np.interp(GRB_z, model.z, model.DL_DH)*model.c_over_H0/1e-5)
    chi = np.sqrt(np.nanmean(((GRB_mu - mu_model)/GRB_err_mu)**2))
    ax.plot(model.z, model.DL_DH*model.c_over_H0/model.z, style,
            label=f'$H_0$={model.H0:.2f};' + r' $\sqrt{\frac{\chi^2}{N}}$' + f'={chi:.2f}')
ax.legend()
ax.set_xlabel('z')

ax.set_xscale('log')
ax.set_xlim(1e-3, 20)
#ax.set_xlim(-.2, 8.2)
ax.set_xlabel('z')

for ax in axes.flatten():
    ax.tick_params(which='both', direction='in')
    ax.grid(alpha=.5)
fig.set_tight_layout(True)
plt.show()
#plt.savefig('DL.pdf')


In [ ]:
def plot_theta_BAO_obs(ax, z, mu, err_mu, **kwargs):
    #scale = 3e5 * z
    scale = z
    y = Dl_Mpc(mu) / scale
    dy_low = y-Dl_Mpc(mu-err_mu) / scale
    dy_high = Dl_Mpc(mu+err_mu) / scale - y
    ax.errorbar(z, y, yerr=(dy_low, dy_high), fmt='.', **kwargs)

In [ ]:
fig_name = 'DA'

plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(10, 6))
axes = fig.subplots(nrows=1, ncols=1, squeeze=False,
                    sharex=True, sharey=True,
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )

ax = axes[0, 0]
ax.set_ylabel(r'$\frac{180}{\pi} \frac{r_S}{D_M(z)}$ = $\theta_{BAO}$ [$^\circ$]')
#ax.set_ylim(-1, 21)
ax.set_yscale('log')
ax.set_ylim(.03, 30)
ax.set_xscale('log')
#ax.set_xlim(.05, 2000)
#ax.set_xlim(-.1, 3.1)

ax.errorbar(theta_z, theta_angle_deg, theta_angle_err, theta_z_err, fmt='k.', label='Nunes et al. (2020)')

'''
for beta in np.linspace(.3, 10, 10):
    model = LwDM_model(1+1/beta**2, -1/3, 0)
    model.fit_theta_BAO(theta_z[1], theta_angle_deg[1], theta_angle_err[1])
    norm = 180/np.pi * model.RS_DH
    ax.plot(model.z, norm/model.DM_DH, 'k-', alpha=.2)
'''

'''
'''
for model_name in models:
    model, style = models[model_name]
    model.fit_theta_BAO(theta_z, theta_angle_deg, theta_angle_err)
    norm = 180/np.pi * model.RS_DH
    ax.plot(model.z, norm/model.DM_DH, style,
            label=f'{model_name}:  '
            + r'$\frac{r_S H_0}{c}$=' + f'{model.RS_DH:.4f};'
            + r' $\sqrt{\frac{\chi^2}{N}}$' + f'={np.sqrt(model.chi2_theta_BAO):.2f};'
            + r' $\theta_{BAO}(z_{CMB})$' + f'={np.interp(z_cmb_model, model.z, norm/model.DM_DH):.2f}$^\circ$'
           )

norm = 180/np.pi * 147.16
ax.plot(model.z, norm/Planck18.angular_diameter_distance(model.z)/(1+model.z), 'r:', label='Planck18')

ax.axvline(1100, c='r', ls='--', label=r'$z_{CMB} = 1100$')
ax.axvline(z_cmb_model, c='k', ls='-', label=r'$z_{CMB}$' + f' = {z_cmb_model:g}')
ax.legend()

for ax in axes.flatten():
    ax.tick_params(which='both', direction='in')
    ax.grid(alpha=.5)
fig.set_tight_layout(True)
plt.show()